In [ ]:
'''
Final Assignment: Data driven approach to laminar and turbulent flow classification

Laminar flow has been characterized for its smooth and linear behavior concerning its flow.
When a non-linearity is introduced, the flow becomes turbulent.
Visually it is possible to distinguish between laminar and turbulent flow by
identifying areas in the flow that become chaotic or non-smooth.
We used a Convolutional Neural Network (CNN) trained on labeled laminar and
turbulent images to automate the process. In this assignment, we take simulated
contours from a flow past cylinder simulation, train it, and test it to see how
effective the CNN is at distinguishing between the laminar and turbulent flow.
The geometry is as follows -
There is a circular cylinder axis along the z-axis(out of the plane),
you have to simulate 2d velocity profile about it.

Objective of CNN model is that given a simulated velocity profile,
it should be able to efficiently predict the nature of fluid flow.
Compare the accuracy of different CNN architectures covered in the class and
based on that choose a model of your dataset.
Then optimize hyperparameters to get accuracy more than 80%

 Deadline to the assignment is : 10th May
'''

In [ ]:
'''
1. create  and preprocess
- 10 laminar/10 turbulent images for dataset
- preprocess - resize, normalise
2. try some cnns
3. optimise hyperparameters (80% accuracy)
4. documentation
'''

In [11]:
# importing libraries

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

In [12]:
# setting a seed to ensure reproducibility

np.random.seed(42)
tf.random.set_seed(42)

In [13]:
# GPU

print("GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

laminar_dir = "/content/drive/MyDrive/Fluid_flow_data/laminar"
laminar_dir_p = "/content/drive/MyDrive/Fluid_flow_data/laminarp"

turbulant_dir = "/content/drive/MyDrive/Fluid_flow_data/turbulent"
turbulant_dir_p = "/content/drive/MyDrive/Fluid_flow_data/turbulantp"

#source_dir = "/content/drive/MyDrive/Fluid_flow_data"
train_dir = "/content/drive/MyDrive/Fluid_flow_data/train"
test_dir = "/content/drive/MyDrive/Fluid_flow_data/test"
val_dir = "/content/drive/MyDrive/Fluid_flow_data/validation"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(laminar_dir_p, exist_ok=True)
os.makedirs(turbulant_dir_p, exist_ok=True)

lamtrain = os.path.join(train_dir, "laminar")
lamtest = os.path.join(test_dir, "laminar")
lamval = os.path.join(val_dir, "laminar")

turtrain = os.path.join(train_dir, "turbulant")
turtest = os.path.join(test_dir, "turbulant")
turval = os.path.join(val_dir, "turbulant")

os.makedirs(lamtrain, exist_ok=True)
os.makedirs(lamtest, exist_ok=True)
os.makedirs(lamval, exist_ok=True)
os.makedirs(turtrain, exist_ok=True)
os.makedirs(turtest, exist_ok=True)
os.makedirs(turval, exist_ok=True)

#train_dir = r"/content/drive/MyDrive/Fluid_flow_data"


GPUs Available:  0


In [14]:
img_width, img_height = 224, 224
num_classes = 2
batchsize = 10

In [15]:
# function to preprocess images
def preprocess_images(source_dir, target_dir):
      for file in os.listdir(source_dir):
          # load image
          img = load_img(os.path.join(source_dir, file), target_size=(img_width, img_height))
          img_array = img_to_array(img)
          img_array /= 255.0  # normalize pixel values
          # move preprocessed image to target directory
          # os.rename(os.path.join(subdir, file), os.path.join(target_dir, subdir.split('/')[-1] + '_' + file))
          os.rename(os.path.join(source_dir, file), os.path.join(target_dir, file))

preprocess_images(laminar_dir, laminar_dir_p)
preprocess_images(turbulant_dir, turbulant_dir_p)

In [16]:
# split data into train, test, and validation
laminar_files = os.listdir(laminar_dir_p)
turbulent_files = os.listdir(turbulant_dir_p)

laminar_train, laminar_test_val = train_test_split(laminar_files, test_size=0.3, random_state=42)
laminar_test, laminar_val = train_test_split(laminar_test_val, test_size=0.5, random_state=42)

turbulent_train, turbulent_test_val = train_test_split(turbulent_files, test_size=0.3, random_state=42)
turbulent_test, turbulent_val = train_test_split(turbulent_test_val, test_size=0.5, random_state=42)


In [17]:
def move_files(source_files, source_dir, dest_dir):
    for file in source_files:
        #os.rename(os.path.join(source_dir, file), os.path.join(dest_dir, file))
        os.rename(os.path.join(source_dir, file), os.path.join(dest_dir, file))

In [18]:
# move files to appropriate directories

move_files(laminar_train, laminar_dir_p, os.path.join(train_dir, "laminar"))
move_files(laminar_test, laminar_dir_p, os.path.join(test_dir, "laminar"))
move_files(laminar_val, laminar_dir_p, os.path.join(val_dir, "laminar"))

move_files(turbulent_train, turbulant_dir_p, os.path.join(train_dir, "turbulant"))
move_files(turbulent_test, turbulant_dir_p, os.path.join(test_dir, "turbulant"))
move_files(turbulent_val, turbulant_dir_p, os.path.join(val_dir, "turbulant"))


In [19]:
# one-hot encoding
# 0 - laminar
# 1 - turbulent

# train
train_files = []
train_files.extend([os.path.join(lamtrain, file) for file in laminar_train])
train_files.extend([os.path.join(turtrain, file) for file in turbulent_train])

train_labels = []
train_labels.extend([0] * len(laminar_train))
train_labels.extend([1] * len(turbulent_train))

train_labels = to_categorical(train_labels, 2)

# test
test_files = []
test_files.extend([os.path.join(lamtest, file) for file in laminar_test])
test_files.extend([os.path.join(turtest, file) for file in turbulent_test])

test_labels = []
test_labels.extend([0] * len(laminar_test))
test_labels.extend([1] * len(turbulent_test))

test_labels = to_categorical(test_labels, 2)

# validation
val_files = []
val_files.extend([os.path.join(lamval, file) for file in laminar_val])
val_files.extend([os.path.join(turval, file) for file in turbulent_val])

val_labels = []
val_labels.extend([0] * len(laminar_val))
val_labels.extend([1] * len(turbulent_val))

val_labels = to_categorical(val_labels,2)


In [20]:
# building the model

model1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# compiling the model
model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [21]:
# Load and preprocess images
train_images = []

for file in train_files:

    img = load_img(file, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    train_images.append(img_array)

# Convert lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

val_images = []

for file in val_files:

    img = load_img(file, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    val_images.append(img_array)

# Convert lists to numpy arrays
val_images = np.array(val_images)
val_labels = np.array(val_labels)


In [22]:
import tensorflow.keras.backend as K
K.clear_session()


In [23]:
# fitting the model
history = model1.fit(train_images, train_labels, batch_size = 15, validation_data=(val_images, val_labels), epochs=5)


Epoch 1/5
14/14 [==============================] - 39s 3s/step - loss: 378.2673 - accuracy: 0.6139 - val_loss: 16.7579 - val_accuracy: 0.6444
Epoch 2/5
14/14 [==============================] - 35s 3s/step - loss: 3.8059 - accuracy: 0.7871 - val_loss: 0.4734 - val_accuracy: 0.7111
Epoch 3/5
14/14 [==============================] - 35s 3s/step - loss: 0.3834 - accuracy: 0.8416 - val_loss: 0.4068 - val_accuracy: 0.8667
Epoch 4/5
14/14 [==============================] - 37s 3s/step - loss: 0.1123 - accuracy: 0.9653 - val_loss: 0.2585 - val_accuracy: 0.8889
Epoch 5/5
14/14 [==============================] - 35s 2s/step - loss: 0.0809 - accuracy: 0.9653 - val_loss: 0.5230 - val_accuracy: 0.9111


In [24]:
test_images = []

for file in test_files:

    img = load_img(file, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    test_images.append(img_array)

# Convert lists to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)


In [25]:
# Evaluating the model
test_loss, test_accuracy = model1.evaluate(test_images, test_labels, batch_size=10)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)

# Predicting classes
predictions = model1.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Generating true classes
true_classes = np.argmax(test_labels, axis=1)

# Generating class labels
class_labels = ["laminar", "turbulent"]

# Generating classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


5/5 [==============================] - 2s 269ms/step - loss: 1.5761e-04 - accuracy: 1.0000
Test Loss: 0.00015760649694129825
Test Accuracy: 100.0
2/2 [==============================] - 1s 353ms/step
              precision    recall  f1-score   support

     laminar       1.00      1.00      1.00        21
   turbulent       1.00      1.00      1.00        23

    accuracy                           1.00        44
   macro avg       1.00      1.00      1.00        44
weighted avg       1.00      1.00      1.00        44



In [26]:
model2 = Sequential([
        Conv2D(96, (11,11), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D(3,3),
        Conv2D(256, (5,5), padding='same', activation='relu'),
        Conv2D(256, (3,3), padding='same', activation='relu'),
        MaxPooling2D(3,3),
        Conv2D(384, (3,3), padding='same', activation='relu'),
        Conv2D(384, (3,3), padding='same', activation='relu'),
        MaxPooling2D(3,3),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])

# compiling the model
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# print(model.summary())


In [27]:
# fitting the model
history = model2.fit(train_images, train_labels, batch_size = 10, validation_data=(val_images, val_labels), epochs=4)


Epoch 1/4
21/21 [==============================] - 294s 14s/step - loss: 137.9366 - accuracy: 0.4802 - val_loss: 1.3150 - val_accuracy: 0.4444
Epoch 2/4
21/21 [==============================] - 300s 14s/step - loss: 0.8753 - accuracy: 0.4406 - val_loss: 0.6966 - val_accuracy: 0.4667
Epoch 3/4
21/21 [==============================] - 294s 14s/step - loss: 0.6964 - accuracy: 0.4455 - val_loss: 0.6936 - val_accuracy: 0.5111
Epoch 4/4
21/21 [==============================] - 296s 14s/step - loss: 0.7083 - accuracy: 0.4505 - val_loss: 0.6933 - val_accuracy: 0.4667


In [29]:
# Evaluating the model
test_loss, test_accuracy = model2.evaluate(test_images, test_labels, batch_size=10)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)

# Predicting classes
predictions = model2.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Generating true classes
true_classes = np.argmax(test_labels, axis=1)

# Generating class labels
class_labels = ["laminar", "turbulent"]

# Generating classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


5/5 [==============================] - 17s 3s/step - loss: 0.6932 - accuracy: 0.4773
Test Loss: 0.6931898593902588
Test Accuracy: 47.72727191448212
2/2 [==============================] - 18s 4s/step
              precision    recall  f1-score   support

     laminar       0.48      1.00      0.65        21
   turbulent       0.00      0.00      0.00        23

    accuracy                           0.48        44
   macro avg       0.24      0.50      0.32        44
weighted avg       0.23      0.48      0.31        44



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
# LeNet5

def LeNet5(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())

    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model


lenet_model = LeNet5((224, 224, 3), 2)
lenet_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [35]:
# fitting the model
history = lenet_model.fit(train_images, train_labels, batch_size = 10, validation_data=(val_images, val_labels), epochs=4)

Epoch 1/4
21/21 [==============================] - 30s 531ms/step - loss: 101.3700 - accuracy: 0.5990 - val_loss: 1.0709 - val_accuracy: 0.6667
Epoch 2/4
21/21 [==============================] - 9s 440ms/step - loss: 0.5487 - accuracy: 0.8416 - val_loss: 0.5332 - val_accuracy: 0.8889
Epoch 3/4
21/21 [==============================] - 10s 481ms/step - loss: 0.1785 - accuracy: 0.9356 - val_loss: 0.5007 - val_accuracy: 0.8889
Epoch 4/4
21/21 [==============================] - 11s 550ms/step - loss: 0.0940 - accuracy: 0.9901 - val_loss: 0.8949 - val_accuracy: 0.8889


In [36]:
# Evaluating the model
test_loss, test_accuracy = lenet_model.evaluate(test_images, test_labels, batch_size=10)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)

# Predicting classes
predictions = model1.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Generating true classes
true_classes = np.argmax(test_labels, axis=1)

# Generating class labels
class_labels = ["laminar", "turbulent"]

# Generating classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

5/5 [==============================] - 1s 121ms/step - loss: 0.4315 - accuracy: 0.8409
Test Loss: 0.4314837157726288
Test Accuracy: 84.09090638160706
2/2 [==============================] - 2s 365ms/step
              precision    recall  f1-score   support

     laminar       1.00      1.00      1.00        21
   turbulent       1.00      1.00      1.00        23

    accuracy                           1.00        44
   macro avg       1.00      1.00      1.00        44
weighted avg       1.00      1.00      1.00        44



In [37]:
# Alexnet

def AlexNet(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    model.add(Flatten())

    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model


alexnet_model = AlexNet((224, 224, 3), 2)
alexnet_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [38]:
# fitting the model
history = alexnet_model.fit(train_images, train_labels, batch_size = 10, validation_data=(val_images, val_labels), epochs=4)

Epoch 1/4
21/21 [==============================] - 27s 1s/step - loss: 19.2408 - accuracy: 0.5050 - val_loss: 0.7057 - val_accuracy: 0.4667
Epoch 2/4
21/21 [==============================] - 26s 1s/step - loss: 0.6928 - accuracy: 0.5099 - val_loss: 0.6418 - val_accuracy: 0.5111
Epoch 3/4
21/21 [==============================] - 24s 1s/step - loss: 0.6750 - accuracy: 0.5545 - val_loss: 0.6554 - val_accuracy: 0.5111
Epoch 4/4
21/21 [==============================] - 25s 1s/step - loss: 0.7118 - accuracy: 0.5842 - val_loss: 0.6923 - val_accuracy: 0.5333


In [39]:
# Evaluating the model
test_loss, test_accuracy = alexnet_model.evaluate(test_images, test_labels, batch_size=10)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)

# Predicting classes
predictions = model1.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Generating true classes
true_classes = np.argmax(test_labels, axis=1)

# Generating class labels
class_labels = ["laminar", "turbulent"]

# Generating classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

5/5 [==============================] - 1s 175ms/step - loss: 0.6944 - accuracy: 0.5000
Test Loss: 0.6944255828857422
Test Accuracy: 50.0
2/2 [==============================] - 1s 292ms/step
              precision    recall  f1-score   support

     laminar       1.00      1.00      1.00        21
   turbulent       1.00      1.00      1.00        23

    accuracy                           1.00        44
   macro avg       1.00      1.00      1.00        44
weighted avg       1.00      1.00      1.00        44



In [41]:
# VGG16

def VGG16(input_shape, num_classes):
    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())

    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model


vgg16_model = VGG16((224, 224, 3), 2)
vgg16_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [42]:
# fitting the model
history = vgg16_model.fit(train_images, train_labels, batch_size = 10, validation_data=(val_images, val_labels), epochs=4)

Epoch 1/4
21/21 [==============================] - 534s 25s/step - loss: 363.0900 - accuracy: 0.5198 - val_loss: 0.7221 - val_accuracy: 0.4667
Epoch 2/4
21/21 [==============================] - 537s 26s/step - loss: 0.7080 - accuracy: 0.5099 - val_loss: 0.6923 - val_accuracy: 0.5333
Epoch 3/4
21/21 [==============================] - 537s 26s/step - loss: 0.6934 - accuracy: 0.5297 - val_loss: 0.6918 - val_accuracy: 0.5333
Epoch 4/4
21/21 [==============================] - 529s 25s/step - loss: 0.6931 - accuracy: 0.5297 - val_loss: 0.6920 - val_accuracy: 0.5333


In [43]:
# Evaluating the model
test_loss, test_accuracy = vgg16_model.evaluate(test_images, test_labels, batch_size=10)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy*100)

# Predicting classes
predictions = model1.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# Generating true classes
true_classes = np.argmax(test_labels, axis=1)

# Generating class labels
class_labels = ["laminar", "turbulent"]

# Generating classification report
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

5/5 [==============================] - 29s 6s/step - loss: 0.6924 - accuracy: 0.5227
Test Loss: 0.6924317479133606
Test Accuracy: 52.272725105285645
2/2 [==============================] - 1s 307ms/step
              precision    recall  f1-score   support

     laminar       1.00      1.00      1.00        21
   turbulent       1.00      1.00      1.00        23

    accuracy                           1.00        44
   macro avg       1.00      1.00      1.00        44
weighted avg       1.00      1.00      1.00        44

